![transformer模型](img/transformer.png) 
<center><big>transformer模型</big></center>

![transformer-encoder模型](img/transformer-encoder.png)
<center><big>transformer-encoder模型</big></center>

In [ ]:
'''
利用transformer左半部分的transformer-encoder作多标签分类
直接将encoder的输出接：一个全连接层 -> 一个sigmoid激活层 作分类模型使用
'''

In [1]:
import os
import re
import pandas as pd

parent_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
path = os.path.join(parent_path, 'data/题库')

def combine_data(data_path):
    """
    把四门科目内的所有文件合并
    """
    r = re.compile(r'\[知识点：\]\n(.*)')  # 用来寻找知识点的正则表达式
    r1 = re.compile(r'纠错复制收藏到空间加入选题篮查看答案解析|\n|知识点：|\s|\[题目\]')  # 简单清洗

    data = []
    for root, dirs, files in os.walk(data_path):
        if files:  # 如果文件夹下有csv文件
            for f in files:
                subject = re.findall('高中_(.{2})', root)[0]
                topic = f.strip('.csv')
                tmp = pd.read_csv(os.path.join(root, f))  # 打开csv文件
                tmp['subject'] = subject  # 主标签：科目
                tmp['topic'] = topic  # 副标签：科目下主题
                tmp['knowledge'] = tmp['item'].apply(
                    lambda x: r.findall(x)[0].replace(',', ' ') if r.findall(x) else '')
                tmp['item'] = tmp['item'].apply(lambda x: r1.sub('', r.sub('', x)))
                data.append(tmp)

    data = pd.concat(data).rename(columns={'item': 'content'}).reset_index(drop=True)
    # 删掉多余的两列
    data.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1, inplace=True)
    return data

In [2]:
df = combine_data(path)
df

,content,subject,topic,knowledge
0,据《左传》记载，春秋后期鲁国大夫季孙氏的家臣阳虎独掌权柄后，标榜要替鲁国国君整肃跋扈的大夫，...,历史,古代史,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度
1,秦始皇统一六国后创制了一套御玺。如任命国家官员，则封印“皇帝之玺”；若任命四夷的官员，则用“...,历史,古代史,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 秦始皇 皇帝制度
2,北宋加强中央集权的主要措施有（）①把主要将领的兵权收归中央②派文官担任地方长官③设置通判监督...,历史,古代史,“重农抑商”政策 郡县制 夏商两代的政治制度 选官、用官制度的变化 中央官制——三公九卿制 ...
3,商朝人崇信各种鬼神，把占卜、祭祀作为与神灵沟通的手段，负责通神事务的是商王和巫师（往往出身贵...,历史,古代史,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度
4,公元963年，北宋政府在江淮地区设置了包括盐业管理，以及控制对茶叶销售的专卖等为主要职责的转...,历史,古代史,“重农抑商”政策 郡县制 夏商两代的政治制度 选官、用官制度的变化 中央官制——三公九卿制 ...
...,...,...,...,...
29808,用纯种的高杆（D）抗锈病（T）小麦与矮杆（d）易染锈病（t）小麦培育矮杆抗锈病小麦新品种的方...,生物,遗传与进化,染色体变异 拉马克的进化学说 生物变异的应用
29809,"下图表示某二倍体生物细胞分裂和受精作用过程中,核DNA含量和染色体数目的变化,正确的是（）A...",生物,遗传与进化,遗传的分子基础 人工授精、试管婴儿等生殖技术 减数第一、二次分裂过程中染色体的行为变化 基因...
29810,下列关于“调查人群中的遗传病”的叙述，错误的是()A.最好选取群体中发病率较高的多基因遗传病...,生物,遗传与进化,不完全显性 人工授精、试管婴儿等生殖技术 生物性污染 避孕的原理和方法 人类遗传病的监测和预防
29811,下图是人类一种遗传病的家系图谱（图中阴影部分表示患者）推测这种病的遗传方式是()A.常染色体...,生物,遗传与进化,不完全显性 人类遗传病的类型及危害 人工授精、试管婴儿等生殖技术 生物性污染 避孕的原理和方法


In [3]:
from collections import Counter

knowledges_point = ' '.join(df['knowledge']).split()
knowledges_point = Counter(knowledges_point)
print('标签个数：{}'.format(len(knowledges_point)))

标签个数：919


In [48]:
knowledges_point.most_common()[:5]

[('人工授精、试管婴儿等生殖技术', 4402),
 ('生物性污染', 4402),
 ('避孕的原理和方法', 4402),
 ('遗传的细胞基础', 2487),
 ('遗传的分子基础', 2455)]

In [17]:
knowledges_point.most_common()[-5:]

[('酶的发现历程', 1),
 ('植物色素的提取', 1),
 ('探究水族箱（或鱼缸）中群落的演替', 1),
 ('证明DNA是主要遗传物质的实验', 1),
 ('生物多样性形成的影响因素', 1)]

In [4]:
# 过滤低频knowledge_point，过滤频率低于样本数的1%的knowledge_point

def filter_extract_label(df, freq=0.01):
    knowledges_point = ' '.join(df['knowledge']).split()
    knowledges_point = Counter(knowledges_point)
    filter_counter = int(df.shape[0] * freq) # 样本数的1%的
    print('过滤出现频率少于{}的knowledge_point'.format(filter_counter))
    filter_knowledge_point = {k for k in knowledges_point if knowledges_point[k] > filter_counter}
    df.knowledge = df.knowledge.apply(lambda x :' '.join([label for label in x.split() if label in filter_knowledge_point]))
    #df['label'] = df[['subject', 'topic', 'knowledge']].apply(lambda x : ' '.join(x), axis=1)
    return df[['knowledge', 'content']]

In [5]:
df = filter_extract_label(df)
df

过滤出现频率少于298的knowledge_point


,knowledge,content
0,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度,据《左传》记载，春秋后期鲁国大夫季孙氏的家臣阳虎独掌权柄后，标榜要替鲁国国君整肃跋扈的大夫，...
1,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度,秦始皇统一六国后创制了一套御玺。如任命国家官员，则封印“皇帝之玺”；若任命四夷的官员，则用“...
2,“重农抑商”政策 郡县制 夏商两代的政治制度 选官、用官制度的变化 中央官制——三公九卿制 ...,北宋加强中央集权的主要措施有（）①把主要将领的兵权收归中央②派文官担任地方长官③设置通判监督...
3,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度,商朝人崇信各种鬼神，把占卜、祭祀作为与神灵沟通的手段，负责通神事务的是商王和巫师（往往出身贵...
4,“重农抑商”政策 郡县制 夏商两代的政治制度 选官、用官制度的变化 中央官制——三公九卿制 ...,公元963年，北宋政府在江淮地区设置了包括盐业管理，以及控制对茶叶销售的专卖等为主要职责的转...
...,...,...
29808,拉马克的进化学说,用纯种的高杆（D）抗锈病（T）小麦与矮杆（d）易染锈病（t）小麦培育矮杆抗锈病小麦新品种的方...
29809,遗传的分子基础 人工授精、试管婴儿等生殖技术 基因的分离规律的实质及应用 生物性污染 减数分...,"下图表示某二倍体生物细胞分裂和受精作用过程中,核DNA含量和染色体数目的变化,正确的是（）A..."
29810,不完全显性 人工授精、试管婴儿等生殖技术 生物性污染 避孕的原理和方法,下列关于“调查人群中的遗传病”的叙述，错误的是()A.最好选取群体中发病率较高的多基因遗传病...
29811,不完全显性 人工授精、试管婴儿等生殖技术 生物性污染 避孕的原理和方法,下图是人类一种遗传病的家系图谱（图中阴影部分表示患者）推测这种病的遗传方式是()A.常染色体...


In [6]:
#过滤后label个数

knowledges_point = ' '.join(df['knowledge']).split()
knowledges_point = Counter(knowledges_point)
print('标签个数：{}'.format(len(knowledges_point)))

标签个数：73


In [6]:
# 加载停词
stopwords_path = os.path.join(os.getcwd(),  'data', 'stopwords.txt')

stopwords_set = set()
with open(stopwords_path, 'r', encoding='utf-8') as f_read:
    for line in f_read:
        stopwords_set.add(line.strip())

print('stop words len :{}'.format(len(stopwords_set)))

stop words len :859


In [7]:
import jieba

def content_preprocess(content):
    # 去标点
    r = re.compile("[^\u4e00-\u9fa5]+|题目")
    content = r.sub("", content)  # 删除所有非汉字字符
    # jieba分词
    words = jieba.cut(content, cut_all=False)
    words = [w for w in words if w not in stopwords_set]
    words = ' '.join(words)
    return words

df.content = df.content.apply(content_preprocess)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.588 seconds.
Prefix dict has been built successfully.
D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
df

,knowledge,content
0,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度,左传 记载 春秋 后期 鲁国 大夫 季孙氏 家臣 阳虎 独掌 权柄 后 标榜 鲁国 国君 整...
1,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度,秦始皇 统一 六国后 创制 一套 御玺 任命 国家 官员 封印 皇帝 之玺 任命 四夷 官员...
2,“重农抑商”政策 郡县制 夏商两代的政治制度 选官、用官制度的变化 中央官制——三公九卿制 ...,北宋 中央集权 措施 将领 兵权 收归 中央 派 文官 担任 地方 长官 设置 通判 监督 ...
3,“重农抑商”政策 郡县制 夏商两代的政治制度 中央官制——三公九卿制 皇帝制度,商朝人 崇信 鬼神 占卜 祭祀 神灵 沟通 手段 负责 通神 事务 商王 巫师 出身 贵族 ...
4,“重农抑商”政策 郡县制 夏商两代的政治制度 选官、用官制度的变化 中央官制——三公九卿制 ...,公元 年 北宋 政府 江淮地区 设置 包括 盐业 管理 控制 茶叶 销售 专卖 主要职责 转...
...,...,...
29808,拉马克的进化学说,纯种 高杆 抗 锈病 小麦 矮杆 易染 锈病 小麦 培育 矮杆 抗 锈病 小麦 新品种 方法...
29809,遗传的分子基础 人工授精、试管婴儿等生殖技术 基因的分离规律的实质及应用 生物性污染 减数分...,下图 二倍体 生物 细胞分裂 受精 作用 过程 中核 含量 染色体 数目 变化 正确 孟德尔...
29810,不完全显性 人工授精、试管婴儿等生殖技术 生物性污染 避孕的原理和方法,调查 人群 中 遗传病 叙述 错误 选取 群体 中 发病率 高 基因 遗传病 患者 家庭成员...
29811,不完全显性 人工授精、试管婴儿等生殖技术 生物性污染 避孕的原理和方法,下图 人类 一种 遗传病 家系 图谱 图中 阴影 患者 推测 病 遗传 方式 常 染色体 显...


In [9]:
# 构建多标签数据
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

mlb = MultiLabelBinarizer()
mlb_result = mlb.fit_transform([df.loc[i, 'knowledge'].split(' ') for i in range(len(df))])
mlb_result = np.delete(mlb_result, 0, axis = 1)
classes_ =np.delete(mlb.classes_, 0)
df_final = pd.concat([df['content'], pd.DataFrame(mlb_result, columns=list(classes_))], axis=1)
df_final

,content,“重农抑商”政策,不完全显性,与细胞分裂有关的细胞器,中央官制——三公九卿制,中心体的结构和功能,人体免疫系统在维持稳态中的作用,人体水盐平衡调节,人体的体温调节,人口增长与人口问题,...,胚胎移植,蛋白质的合成,血糖平衡的调节,走进细胞,选官、用官制度的变化,遗传的分子基础,遗传的细胞基础,避孕的原理和方法,郡县制,高尔基体的结构和功能
0,左传 记载 春秋 后期 鲁国 大夫 季孙氏 家臣 阳虎 独掌 权柄 后 标榜 鲁国 国君 整...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,秦始皇 统一 六国后 创制 一套 御玺 任命 国家 官员 封印 皇帝 之玺 任命 四夷 官员...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,北宋 中央集权 措施 将领 兵权 收归 中央 派 文官 担任 地方 长官 设置 通判 监督 ...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,商朝人 崇信 鬼神 占卜 祭祀 神灵 沟通 手段 负责 通神 事务 商王 巫师 出身 贵族 ...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,公元 年 北宋 政府 江淮地区 设置 包括 盐业 管理 控制 茶叶 销售 专卖 主要职责 转...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29808,纯种 高杆 抗 锈病 小麦 矮杆 易染 锈病 小麦 培育 矮杆 抗 锈病 小麦 新品种 方法...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29809,下图 二倍体 生物 细胞分裂 受精 作用 过程 中核 含量 染色体 数目 变化 正确 孟德尔...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
29810,调查 人群 中 遗传病 叙述 错误 选取 群体 中 发病率 高 基因 遗传病 患者 家庭成员...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
29811,下图 人类 一种 遗传病 家系 图谱 图中 阴影 患者 推测 病 遗传 方式 常 染色体 显...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
import pickle

# 保存知识点标签
words_path = os.path.join(os.getcwd(), 'knowledge_points.pkl')
with open(words_path, 'wb') as f_words:
    pickle.dump(list(classes_), f_words)
    
# 保存训练数据
train_data_path = os.path.join(os.getcwd(), 'train_data.pkl')
with open(train_data_path, 'wb') as f_train:
    pickle.dump(df_final, f_train)

In [2]:
import pickle
import os

import pickle
import os

# 加载知识点标签
knowledge_points_path = os.path.join(os.getcwd(), "knowledge_points.pkl")
with open(knowledge_points_path, 'rb') as f_words:
    knowledge_points = pickle.load(f_words)
    
# 加载训练数据
train_data_path = os.path.join(os.getcwd(), "train_data.pkl")
with open(train_data_path, 'rb') as f_train:
    df_final = pickle.load(f_train)
df_final

,content,“重农抑商”政策,不完全显性,与细胞分裂有关的细胞器,中央官制——三公九卿制,中心体的结构和功能,人体免疫系统在维持稳态中的作用,人体水盐平衡调节,人体的体温调节,人口增长与人口问题,...,胚胎移植,蛋白质的合成,血糖平衡的调节,走进细胞,选官、用官制度的变化,遗传的分子基础,遗传的细胞基础,避孕的原理和方法,郡县制,高尔基体的结构和功能
0,左传 记载 春秋 后期 鲁国 大夫 季孙氏 家臣 阳虎 独掌 权柄 后 标榜 鲁国 国君 整...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,秦始皇 统一 六国后 创制 一套 御玺 任命 国家 官员 封印 皇帝 之玺 任命 四夷 官员...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,北宋 中央集权 措施 将领 兵权 收归 中央 派 文官 担任 地方 长官 设置 通判 监督 ...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,商朝人 崇信 鬼神 占卜 祭祀 神灵 沟通 手段 负责 通神 事务 商王 巫师 出身 贵族 ...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,公元 年 北宋 政府 江淮地区 设置 包括 盐业 管理 控制 茶叶 销售 专卖 主要职责 转...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29808,纯种 高杆 抗 锈病 小麦 矮杆 易染 锈病 小麦 培育 矮杆 抗 锈病 小麦 新品种 方法...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29809,下图 二倍体 生物 细胞分裂 受精 作用 过程 中核 含量 染色体 数目 变化 正确 孟德尔...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
29810,调查 人群 中 遗传病 叙述 错误 选取 群体 中 发病率 高 基因 遗传病 患者 家庭成员...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
29811,下图 人类 一种 遗传病 家系 图谱 图中 阴影 患者 推测 病 遗传 方式 常 染色体 显...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
import os
import torch
from torchtext import data,datasets
from torchtext.data import Iterator, BucketIterator
from torchtext.vocab import Vectors
from torch import nn,optim
import torch.nn.functional as F
import pandas as pd
import pickle

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 按字分    
tokenize =lambda x: x.split(' ')

TEXT = data.Field(
                    sequential=True,
                    tokenize=tokenize,
                    lower=False,
                    use_vocab=True,
                    pad_token='<pad>',
                    unk_token='<unk>',
                    batch_first=True,
                    fix_length=200)

LABEL = data.Field(
                    sequential=False,
                    use_vocab=False,
                    batch_first=True,
                    )

# 获取训练或测试数据集
def get_dataset(csv_data, text_field, label_field, test=False):
    fields = [('id', None), ('text', text_field), ('label', label_field)]
    examples = []
    if test: #测试集，不加载label
        for text in csv_data['content']:
            examples.append(data.Example.fromlist([None, text, None], fields))
    else: # 训练集
        for i in range(len(csv_data)):
            sample = csv_data.loc[i]
            text = sample['content']
            label = [v for v in map(int, sample[knowledge_points])]
            examples.append(data.Example.fromlist([None, text, label], fields))
    return examples, fields

train_examples,train_fields = get_dataset(df_final, TEXT, LABEL)

train = data.Dataset(train_examples, train_fields)
# 预训练数据
#pretrained_embedding = os.path.join(os.getcwd(), 'sgns.sogou.char')
#vectors = Vectors(name=pretrained_embedding)
# 构建词典
#TEXT.build_vocab(train, min_freq=1, vectors = vectors)

TEXT.build_vocab(train, min_freq=1)
words_path = os.path.join(os.getcwd(), 'words.pkl')
with open(words_path, 'wb') as f_words:
    pickle.dump(TEXT.vocab, f_words)
    
print('process done!')

process done!


In [6]:
len(TEXT.vocab.itos)

70057

In [5]:
import random

# 划分训练与验证集，一个问题，利用random_split进行数据集划分后，会丢失fields属性
train_set, val_set = train.split(split_ratio=0.95, random_state=random.seed(1))

BATCH_SIZE = 64
# 生成训练与验证集的迭代器
train_iterator, val_iterator = data.BucketIterator.splits(
    (train_set, val_set),
    batch_size=BATCH_SIZE,
    #shuffle=True,
    # device=device,
    sort_within_batch=False,
    sort_key=lambda x:len(x.text)
)

'''
train_iter = BucketIterator(
                            dataset=train_set,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            sort_within_batch=False)
val_iter = BucketIterator(
                            dataset=val_set,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            sort_within_batch=False)
'''
print('build dataset done!')

build dataset done!


In [6]:
print(len(train_set))
print(len(train_iterator))

28322
443


In [31]:
print(len(train.examples))
print(vars(train.examples[0]))
print(vars(train.examples[1]))

29813
{'text': ['左传', '记载', '春秋', '后期', '鲁国', '大夫', '季孙氏', '家臣', '阳虎', '独掌', '权柄', '后', '标榜', '鲁国', '国君', '整肃', '跋扈', '大夫', '此举', '得不到', '知礼', '之士', '赞成', '反而', '批评', '此举', '挑战', '宗法制度', '损害', '大夫', '利益', '冲击', '天子', '权威', '不', '符合', '周礼', '次数', '阳虎', '身份', '鲁国', '大夫', '季孙氏', '家臣', '周礼', '效忠', '季孙氏', '标榜', '鲁国', '国君', '整肃', '大夫', '僭', '越', '批评', '违背', '周礼', '选择项', '宗法制度', '血缘', '核心', '故项', '与此无关', '排除', '项', '题意', '无关', '排除', '材料', '事件', '涉及', '鲁国', '国内', '周天子', '权威', '无关', '排除', '项'], 'label': [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]}
{'text': ['秦始皇', '统一', '六国后', '创制', '一套', '御玺', '任命', '国家', '官员', '封印', '皇帝', '之玺', '任命', '四夷', '官员', '天子', '之玺', '信玺', '用于', '国内', '四夷', '用兵', '事宜', '行玺', '皇帝', '外', '巡时', '随身携带', '材料', '皇帝', '处于', '至高无上', '地位', '秦朝', '内外', '两种', '系统', '国事', '秦朝', '实行', '中央集权', '体制', '三公九卿', '制

In [32]:
print(len(val_set))
print(len(val_iterator))


1491
12


In [15]:
for i, batch in enumerate(val_iterator):
    train_text = batch.text
    train_label = batch.label
    print(train_text)
    for trai in train_text:
        print(trai)
    print()
    print(train_label)
    break

tensor([[    5,     1,     1,  ...,     1,     1,     1],
        [    5,     1,     1,  ...,     1,     1,     1],
        [16827, 50809,     9,  ...,     1,     1,     1],
        ...,
        [21678,    48,  2581,  ...,     1,     1,     1],
        [13297, 19244,  1205,  ...,     1,     1,     1],
        [  267, 10205, 26392,  ...,     1,     1,     1]])
tensor([5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

tensor([  748,    18,  9053,  1126,  2207,   172,  5045,  2488,  3235, 18747,
         3235,  1282,  7271,  1205,  3235,  1112,  7271,  1508,  3235,     5,
         2893,  5902,  4534,    65,  1282,  7272,  1205, 10259,  1126,    85,
         2207,    85,  1127,   408,   748,   139,    21,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

            1,     1,     1,     1,     1,     1,     1,     1,     1,     1])
tensor([  83, 4685, 9585,   18,  621,    2, 1988,  211,  538, 8148,  379,  468,
          94,   14,  297,   94,  317,   94,  297,   71,   94,   71,   94,    5,
          12,   83, 4685, 9585,  122,  621,    2, 1988,  211,  297,   94, 1988,
         211,   44,  445,  538, 8148,  317,   94,  379,  468,   94,   14,  297,
         317,   94,   19,   15,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    

tensor([47186,  4170,  6526,   849,  4710,    17,  9303,    33,  2229,  2102,
            2,   522,   863,   216,  5228,  2080,  7036,  7922, 10367, 13080,
         1439, 26149, 26561,  1439, 10357,  7036,  7922,  6604,   259,  2395,
           73,     5,  7036,  7922, 10367,  1669, 39028,   917, 13080,  1439,
        26149, 26561,  1439, 10357, 25942, 28023,   308,  5028,  7036,  7922,
         6604,  4370,   259,  2395,    73,   370,  5028,   853,   829,  1135,
          853,     9,    61,   146,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [7]:
'''
    1.输入是序列中token的embedding与位置embedding
    2.token的embedding与其位置embedding相加，得到一个vector(这个向量融合了token与position信息)
    3.在2之前，token的embedding乘上一个scale(防止点积变大，造成梯度过小)向量[sqrt(emb_dim)]，这个假设为了减少embedding中的变化，没有这个scale，很难稳定的去训练model。
    4.加入dropout
    5.通过N个encoder layer，得到Z。此输出Z被传入一个全连接层作分类。
    src_mask对于非<pad>值为1,<pad>为0。为了计算attention而遮挡<pad>这个无意义的token。与source 句子shape一致。
'''
class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, output_dim, emb_dim, n_layers, n_heads, pf_dim, dropout, position_length, pad_idx):
        super(TransformerEncoder, self).__init__()
        
        self.pad_idx = pad_idx
        self.scale = torch.sqrt(torch.FloatTensor([emb_dim])).to(DEVICE)

        # 词的embedding
        self.token_embedding = nn.Embedding(input_dim, emb_dim)
        # 对词的位置进行embedding
        self.position_embedding = nn.Embedding(position_length, emb_dim)
        # encoder层，有几个encoder层，每个encoder有几个head
        self.layers = nn.ModuleList([EncoderLayer(emb_dim, n_heads, pf_dim, dropout) for _ in range(n_layers)])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(emb_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def mask_src_mask(self, src):
        # src=[batch_size, src_len]

        # src_mask=[batch_size, 1, 1, src_len]
        src_mask = (src != self.pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask
    
    def forward(self, src):
        # src=[batch_size, seq_len]
        # src_mask=[batch_size, 1, 1, seq_len]
        src_mask = self.mask_src_mask(src)
        
        batch_size = src.shape[0]
        src_len = src.shape[1]

        # 构建位置tensor -> [batch_size, seq_len]，位置序号从(0)开始到(src_len-1)
        position = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(DEVICE)

        # 对词和其位置进行embedding -> [batch_size,seq_len,embdim]
        token_embeded = self.token_embedding(src) * self.scale
        position_embeded = self.position_embedding(position)

        # 对词和其位置的embedding进行按元素加和 -> [batch_size, seq_len, embdim]
        src = self.dropout(token_embeded + position_embeded)

        for layer in self.layers:
            src = layer(src, src_mask)

        # [batch_size, seq_len, emb_dim] -> [batch_size, output_dim]
        src = src.permute(0, 2, 1)
        src = torch.sum(src, dim=-1)
        src = self.fc(src)
        src = self.sigmoid(src)
        return src

'''
encoder layers：
    1.将src与src_mask传入多头attention层(multi-head attention)
    2.dropout
    3.使用残差连接后传入layer-norm层(输入+输出后送入norm)后得到的输出
    4.输出通过前馈网络feedforward层
    5.dropout
    6.一个残差连接后传入layer-norm层后得到的输出喂给下一层
    注意：
        layer之间不共享参数
        多头注意力层用到的是多个自注意力层self-attention
'''
class EncoderLayer(nn.Module):
    def __init__(self, emb_dim, n_heads, pf_dim, dropout):
        super(EncoderLayer, self).__init__()
        # 注意力层后的layernorm
        self.self_attn_layer_norm = nn.LayerNorm(emb_dim)
        # 前馈网络层后的layernorm
        self.ff_layer_norm = nn.LayerNorm(emb_dim)
        # 多头注意力层
        self.self_attention = MultiHeadAttentionLayer(emb_dim, n_heads, dropout)
        # 前馈层
        self.feedforward = FeedforwardLayer(emb_dim, pf_dim, dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):
        #src=[batch_size, seq_len, emb_dim]
        #src_mask=[batch_size, 1, 1, seq_len]

        # self-attention
        # _src=[batch size, query_len, emb_dim]
        _src, _ = self.self_attention(src, src, src, src_mask)

        # dropout, 残差连接以及layer-norm
        # src=[batch_size, seq_len, emb_dim]
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        # 前馈网络
        # _src=[batch_size, seq_len, emb_dim]
        _src = self.feedforward(src)

        # dropout, 残差连接以及layer-norm
        # src=[batch_size, seq_len, emb_dim]
        src = self.ff_layer_norm(src + self.dropout(_src))

        return src
'''
多头注意力层的计算:
    1.q,k,v的计算是通过线性层fc_q,fc_k,fc_v
    2.对query,key,value的emb_dim split成n_heads
    3.通过计算Q*K/scale计算energy
    4.利用mask遮掩不需要关注的token
    5.利用softmax与dropout
    6.5的结果与V矩阵相乘
    7.最后通过一个前馈fc_o输出结果
注意:Q,K,V的长度一致
'''
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, emb_dim, n_heads, dropout):
        super(MultiHeadAttentionLayer, self).__init__()
        assert emb_dim % n_heads == 0
        self.emb_dim = emb_dim
        self.n_heads = n_heads
        self.head_dim = emb_dim//n_heads

        self.fc_q = nn.Linear(emb_dim, emb_dim)
        self.fc_k = nn.Linear(emb_dim, emb_dim)
        self.fc_v = nn.Linear(emb_dim, emb_dim)

        self.fc_o = nn.Linear(emb_dim, emb_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(DEVICE)

    def forward(self, query, key, value, mask=None):
        # query=[batch_size, query_len, emb_dim]
        # key=[batch_size, key_len, emb_dim]
        # value=[batch_size, value_len, emb_dim]
        batch_size = query.shape[0]

        # Q=[batch_size, query_len, emb_dim]
        # K=[batch_size, key_len, emb_dim]
        # V=[batch_size, value_len, emb_dim]
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        '''
        view与reshape的异同：
        
        torch的view()与reshape()方法都可以用来重塑tensor的shape，区别就是使用的条件不一样。view()方法只适用于满足连续性条件的tensor，并且该操作不会开辟新的内存空间，
        只是产生了对原存储空间的一个新别称和引用，返回值是视图。而reshape()方法的返回值既可以是视图，也可以是副本，当满足连续性条件时返回view，
        否则返回副本[ 此时等价于先调用contiguous()方法在使用view() ]。因此当不确能否使用view时，可以使用reshape。如果只是想简单地重塑一个tensor的shape，
        那么就是用reshape，但是如果需要考虑内存的开销而且要确保重塑后的tensor与之前的tensor共享存储空间，那就使用view()。
        '''

        # Q=[batch_size, n_heads, query_len, head_dim]
        # K=[batch_size, n_heads, key_len, head_dim]
        # V=[batch_size, n_heads, value_len, head_dim]
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        # 注意力打分矩阵 [batch_size, n_heads, query_len, head_dim] * [batch_size, n_heads, head_dim, key_len] = [batch_size, n_heads, query_len, key_len]
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        # [batch_size, n_heads, query_len, key_len]
        attention = torch.softmax(energy , dim = -1)

        # [batch_size, n_heads, query_len, key_len]*[batch_size, n_heads, value_len, head_dim]=[batch_size, n_heads, query_len, head_dim]
        x = torch.matmul(self.dropout(attention), V)

        # [batch_size, query_len, n_heads, head_dim]
        x = x.permute(0, 2, 1, 3).contiguous()

        # [batch_size, query_len, emb_dim]
        x = x.view(batch_size, -1, self.emb_dim)

        # [batch_size, query_len, emb_dim]
        x = self.fc_o(x)

        return x, attention

'''
前馈层
'''
class FeedforwardLayer(nn.Module):
    def __init__(self, emb_dim, pf_dim, dropout):
        super(FeedforwardLayer, self).__init__()
        self.fc_1 = nn.Linear(emb_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, emb_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x=[batch_size, seq_len, emb_dim]

        # x=[batch_size, seq_len, pf_dim]
        x = self.dropout(torch.relu(self.fc_1(x)))

        # x=[batch_size, seq_len, emb_dim]
        x = self.fc_2(x)

        return x

In [8]:

'''
评估
'''
def evaluate(model, criterion):
    model.eval()  # 评估模型，切断dropout与batchnorm
    epoch_loss = 0
    with torch.no_grad():  # 不更新梯度
        for i, batch in enumerate(val_iterator):
            train_text = batch.text  
            train_label = batch.label
            train_label = train_label.float()

            train_text = train_text.to(DEVICE)
            train_label = train_label.to(DEVICE)

            out = model(train_text)
            loss = criterion(out, train_label)
            epoch_loss += float(loss.item())
    print('evaluate loss:{}'.format(epoch_loss/len(val_iterator)))
    
#对所有模块和子模块进行权重初始化
def init_weights(model):
    for name,param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [11]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler

writer = SummaryWriter(os.getcwd()+'/log', comment='transformer-encoder')

# 训练
input_dim = len(TEXT.vocab) 
output_dim = 73 # 共73个知识点标签
emb_dim = 256
n_layers = 3
n_heads = 8
pf_dim = 512
dropout = 0.1
position_length = 200

# <pad>
pad_index = TEXT.vocab.stoi[TEXT.pad_token]

# 构建model
model = TransformerEncoder(input_dim, output_dim, emb_dim, n_layers, n_heads, pf_dim, dropout, position_length, pad_index).to(DEVICE)
#初始化权重
model.apply(init_weights)
# 利用预训练模型初始化embedding，requires_grad=True，可以fine-tune
# model.embedding.weight.data.copy_(TEXT.vocab.vectors)

# 优化和损失
# optimizer = torch.optim.Adam(model.parameters(),lr=0.0001, weight_decay=0.01)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)#, momentum=0.9)#, nesterov=True)
#optimizer = torch.optim.RMSprop(model.parameters(),lr=0.01,alpha=0.99)

# 定义lr衰减
#scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.1)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.BCELoss()

with writer:
    for iter in range(30):
        # 训练模式
        model.train()
        for i, batch in enumerate(train_iterator):
            train_text = batch.text
            train_label = batch.label
            train_label = train_label.float()
            
            train_text = train_text.to(DEVICE)
            train_label = train_label.to(DEVICE)
            
            out = model(train_text)
            loss = criterion(out, train_label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (iter+1) % 5 == 0:
                    print ('iter [{}/{}], Loss: {:.4f}'.format(iter+1, 30, loss.item()))
            #writer.add_graph(model, input_to_model=train_text,verbose=False)
            writer.add_scalar('loss',loss.item(),global_step=iter+1)
        #scheduler.step()
        evaluate(model, criterion)
    writer.flush()
    writer.close()
            
model_path = os.path.join(os.getcwd(), "model.h5")
torch.save(model.state_dict(), model_path)

print('train model done!')

evaluate loss:0.13051100199421248
evaluate loss:0.12831985981514057
evaluate loss:0.1270109962982436
evaluate loss:0.12592634682854018
iter [5/30], Loss: 0.1256
iter [5/30], Loss: 0.1417
iter [5/30], Loss: 0.1200
iter [5/30], Loss: 0.1241
iter [5/30], Loss: 0.1354
iter [5/30], Loss: 0.1338
iter [5/30], Loss: 0.1214
iter [5/30], Loss: 0.1221
iter [5/30], Loss: 0.1218
iter [5/30], Loss: 0.1122
iter [5/30], Loss: 0.1222
iter [5/30], Loss: 0.1195
iter [5/30], Loss: 0.1352
iter [5/30], Loss: 0.1284
iter [5/30], Loss: 0.1216
iter [5/30], Loss: 0.1230
iter [5/30], Loss: 0.1405
iter [5/30], Loss: 0.1114
iter [5/30], Loss: 0.1385
iter [5/30], Loss: 0.1104
iter [5/30], Loss: 0.1234
iter [5/30], Loss: 0.1256
iter [5/30], Loss: 0.1337
iter [5/30], Loss: 0.1311
iter [5/30], Loss: 0.1195
iter [5/30], Loss: 0.1191
iter [5/30], Loss: 0.1393
iter [5/30], Loss: 0.1248
iter [5/30], Loss: 0.1319
iter [5/30], Loss: 0.1302
iter [5/30], Loss: 0.1262
iter [5/30], Loss: 0.1126
iter [5/30], Loss: 0.1347
iter [5

iter [5/30], Loss: 0.1311
iter [5/30], Loss: 0.1288
iter [5/30], Loss: 0.1268
iter [5/30], Loss: 0.1130
iter [5/30], Loss: 0.1445
iter [5/30], Loss: 0.0943
iter [5/30], Loss: 0.1288
iter [5/30], Loss: 0.1139
iter [5/30], Loss: 0.1342
iter [5/30], Loss: 0.1182
iter [5/30], Loss: 0.1441
iter [5/30], Loss: 0.1215
iter [5/30], Loss: 0.1135
iter [5/30], Loss: 0.1276
iter [5/30], Loss: 0.1414
iter [5/30], Loss: 0.1198
iter [5/30], Loss: 0.1171
iter [5/30], Loss: 0.1106
iter [5/30], Loss: 0.1376
iter [5/30], Loss: 0.1419
iter [5/30], Loss: 0.1346
iter [5/30], Loss: 0.1256
iter [5/30], Loss: 0.1566
iter [5/30], Loss: 0.1168
iter [5/30], Loss: 0.1318
iter [5/30], Loss: 0.1362
iter [5/30], Loss: 0.1222
iter [5/30], Loss: 0.1315
iter [5/30], Loss: 0.1076
iter [5/30], Loss: 0.1240
iter [5/30], Loss: 0.1047
iter [5/30], Loss: 0.1168
iter [5/30], Loss: 0.1066
iter [5/30], Loss: 0.1125
iter [5/30], Loss: 0.1061
iter [5/30], Loss: 0.1220
iter [5/30], Loss: 0.1270
iter [5/30], Loss: 0.1052
iter [5/30],

iter [10/30], Loss: 0.0956
iter [10/30], Loss: 0.1059
iter [10/30], Loss: 0.1207
iter [10/30], Loss: 0.1059
iter [10/30], Loss: 0.0998
iter [10/30], Loss: 0.0986
iter [10/30], Loss: 0.0860
iter [10/30], Loss: 0.1168
iter [10/30], Loss: 0.0800
iter [10/30], Loss: 0.0805
iter [10/30], Loss: 0.1089
iter [10/30], Loss: 0.0937
iter [10/30], Loss: 0.0963
iter [10/30], Loss: 0.1246
iter [10/30], Loss: 0.1109
iter [10/30], Loss: 0.1105
iter [10/30], Loss: 0.1033
iter [10/30], Loss: 0.0991
iter [10/30], Loss: 0.1001
iter [10/30], Loss: 0.1037
iter [10/30], Loss: 0.0920
iter [10/30], Loss: 0.1039
iter [10/30], Loss: 0.1193
iter [10/30], Loss: 0.1102
iter [10/30], Loss: 0.1067
iter [10/30], Loss: 0.1079
iter [10/30], Loss: 0.0899
iter [10/30], Loss: 0.1037
iter [10/30], Loss: 0.1048
iter [10/30], Loss: 0.1035
iter [10/30], Loss: 0.0969
iter [10/30], Loss: 0.1032
iter [10/30], Loss: 0.0908
iter [10/30], Loss: 0.1062
iter [10/30], Loss: 0.0920
iter [10/30], Loss: 0.0977
iter [10/30], Loss: 0.1141
i

iter [15/30], Loss: 0.0902
iter [15/30], Loss: 0.0929
iter [15/30], Loss: 0.0928
iter [15/30], Loss: 0.1109
iter [15/30], Loss: 0.0825
iter [15/30], Loss: 0.0946
iter [15/30], Loss: 0.0885
iter [15/30], Loss: 0.0920
iter [15/30], Loss: 0.0848
iter [15/30], Loss: 0.0904
iter [15/30], Loss: 0.0736
iter [15/30], Loss: 0.0794
iter [15/30], Loss: 0.0764
iter [15/30], Loss: 0.0971
iter [15/30], Loss: 0.0660
iter [15/30], Loss: 0.0923
iter [15/30], Loss: 0.0870
iter [15/30], Loss: 0.0874
iter [15/30], Loss: 0.0704
iter [15/30], Loss: 0.0995
iter [15/30], Loss: 0.0776
iter [15/30], Loss: 0.0829
iter [15/30], Loss: 0.0815
iter [15/30], Loss: 0.0749
iter [15/30], Loss: 0.0996
iter [15/30], Loss: 0.0781
iter [15/30], Loss: 0.1080
iter [15/30], Loss: 0.0831
iter [15/30], Loss: 0.0939
iter [15/30], Loss: 0.0815
iter [15/30], Loss: 0.0962
iter [15/30], Loss: 0.0939
iter [15/30], Loss: 0.0776
iter [15/30], Loss: 0.0717
iter [15/30], Loss: 0.0950
iter [15/30], Loss: 0.0863
iter [15/30], Loss: 0.0709
i

iter [15/30], Loss: 0.1088
iter [15/30], Loss: 0.0843
iter [15/30], Loss: 0.0765
iter [15/30], Loss: 0.0742
iter [15/30], Loss: 0.0838
iter [15/30], Loss: 0.0682
iter [15/30], Loss: 0.0784
iter [15/30], Loss: 0.0814
iter [15/30], Loss: 0.1018
iter [15/30], Loss: 0.0935
iter [15/30], Loss: 0.0865
iter [15/30], Loss: 0.0753
iter [15/30], Loss: 0.0856
iter [15/30], Loss: 0.0811
iter [15/30], Loss: 0.0879
iter [15/30], Loss: 0.0838
iter [15/30], Loss: 0.0724
iter [15/30], Loss: 0.0745
iter [15/30], Loss: 0.0858
iter [15/30], Loss: 0.0965
iter [15/30], Loss: 0.0892
iter [15/30], Loss: 0.0862
iter [15/30], Loss: 0.0605
iter [15/30], Loss: 0.0813
iter [15/30], Loss: 0.0696
iter [15/30], Loss: 0.0747
iter [15/30], Loss: 0.0974
iter [15/30], Loss: 0.0940
iter [15/30], Loss: 0.0706
iter [15/30], Loss: 0.0735
iter [15/30], Loss: 0.0792
iter [15/30], Loss: 0.0825
iter [15/30], Loss: 0.0746
iter [15/30], Loss: 0.0843
iter [15/30], Loss: 0.0915
iter [15/30], Loss: 0.0960
iter [15/30], Loss: 0.0877
i

iter [20/30], Loss: 0.0732
iter [20/30], Loss: 0.0895
iter [20/30], Loss: 0.0654
iter [20/30], Loss: 0.0877
iter [20/30], Loss: 0.0732
iter [20/30], Loss: 0.0769
iter [20/30], Loss: 0.0752
iter [20/30], Loss: 0.0684
iter [20/30], Loss: 0.0830
iter [20/30], Loss: 0.0654
iter [20/30], Loss: 0.0791
iter [20/30], Loss: 0.0735
iter [20/30], Loss: 0.0867
iter [20/30], Loss: 0.0781
iter [20/30], Loss: 0.0797
iter [20/30], Loss: 0.0822
iter [20/30], Loss: 0.0739
iter [20/30], Loss: 0.0690
iter [20/30], Loss: 0.0750
iter [20/30], Loss: 0.0578
iter [20/30], Loss: 0.0773
iter [20/30], Loss: 0.0742
iter [20/30], Loss: 0.0668
iter [20/30], Loss: 0.0736
iter [20/30], Loss: 0.0752
iter [20/30], Loss: 0.0737
iter [20/30], Loss: 0.0760
iter [20/30], Loss: 0.0928
iter [20/30], Loss: 0.0772
iter [20/30], Loss: 0.0942
iter [20/30], Loss: 0.0907
iter [20/30], Loss: 0.0647
iter [20/30], Loss: 0.0734
iter [20/30], Loss: 0.0784
iter [20/30], Loss: 0.0833
iter [20/30], Loss: 0.0733
iter [20/30], Loss: 0.0715
i

iter [25/30], Loss: 0.0566
iter [25/30], Loss: 0.0683
iter [25/30], Loss: 0.0794
iter [25/30], Loss: 0.0805
iter [25/30], Loss: 0.0806
iter [25/30], Loss: 0.0706
iter [25/30], Loss: 0.0684
iter [25/30], Loss: 0.0634
iter [25/30], Loss: 0.0820
iter [25/30], Loss: 0.0714
iter [25/30], Loss: 0.0691
iter [25/30], Loss: 0.0495
iter [25/30], Loss: 0.0515
iter [25/30], Loss: 0.0672
iter [25/30], Loss: 0.0866
iter [25/30], Loss: 0.0642
iter [25/30], Loss: 0.0659
iter [25/30], Loss: 0.0782
iter [25/30], Loss: 0.1055
iter [25/30], Loss: 0.0771
iter [25/30], Loss: 0.0753
iter [25/30], Loss: 0.0683
iter [25/30], Loss: 0.0637
iter [25/30], Loss: 0.0673
iter [25/30], Loss: 0.0574
iter [25/30], Loss: 0.0602
iter [25/30], Loss: 0.0756
iter [25/30], Loss: 0.0611
iter [25/30], Loss: 0.0693
iter [25/30], Loss: 0.0681
iter [25/30], Loss: 0.0695
iter [25/30], Loss: 0.0562
iter [25/30], Loss: 0.0668
iter [25/30], Loss: 0.0657
iter [25/30], Loss: 0.0660
iter [25/30], Loss: 0.0578
iter [25/30], Loss: 0.0556
i

iter [25/30], Loss: 0.0570
iter [25/30], Loss: 0.0554
iter [25/30], Loss: 0.0656
iter [25/30], Loss: 0.0643
iter [25/30], Loss: 0.0727
iter [25/30], Loss: 0.0632
iter [25/30], Loss: 0.0592
iter [25/30], Loss: 0.0760
iter [25/30], Loss: 0.0773
iter [25/30], Loss: 0.0576
iter [25/30], Loss: 0.0741
iter [25/30], Loss: 0.0550
iter [25/30], Loss: 0.0778
iter [25/30], Loss: 0.0694
iter [25/30], Loss: 0.0653
iter [25/30], Loss: 0.0493
iter [25/30], Loss: 0.0641
iter [25/30], Loss: 0.0681
iter [25/30], Loss: 0.0766
iter [25/30], Loss: 0.0681
iter [25/30], Loss: 0.0635
iter [25/30], Loss: 0.0580
iter [25/30], Loss: 0.0585
iter [25/30], Loss: 0.0532
iter [25/30], Loss: 0.0540
iter [25/30], Loss: 0.0656
iter [25/30], Loss: 0.0698
iter [25/30], Loss: 0.0591
iter [25/30], Loss: 0.0744
iter [25/30], Loss: 0.0584
iter [25/30], Loss: 0.0765
iter [25/30], Loss: 0.0795
iter [25/30], Loss: 0.0679
iter [25/30], Loss: 0.0512
iter [25/30], Loss: 0.0621
iter [25/30], Loss: 0.0691
iter [25/30], Loss: 0.0690
i

iter [30/30], Loss: 0.0526
iter [30/30], Loss: 0.0659
iter [30/30], Loss: 0.0503
iter [30/30], Loss: 0.0530
iter [30/30], Loss: 0.0579
iter [30/30], Loss: 0.0610
iter [30/30], Loss: 0.0499
iter [30/30], Loss: 0.0632
iter [30/30], Loss: 0.0702
iter [30/30], Loss: 0.0550
iter [30/30], Loss: 0.0529
iter [30/30], Loss: 0.0728
iter [30/30], Loss: 0.0496
iter [30/30], Loss: 0.0659
iter [30/30], Loss: 0.0663
iter [30/30], Loss: 0.0561
iter [30/30], Loss: 0.0520
iter [30/30], Loss: 0.0530
iter [30/30], Loss: 0.0557
iter [30/30], Loss: 0.0677
iter [30/30], Loss: 0.0754
iter [30/30], Loss: 0.0522
iter [30/30], Loss: 0.0487
iter [30/30], Loss: 0.0614
iter [30/30], Loss: 0.0649
iter [30/30], Loss: 0.0506
iter [30/30], Loss: 0.0653
iter [30/30], Loss: 0.0563
iter [30/30], Loss: 0.0647
iter [30/30], Loss: 0.0596
iter [30/30], Loss: 0.0702
iter [30/30], Loss: 0.0643
iter [30/30], Loss: 0.0707
iter [30/30], Loss: 0.0743
iter [30/30], Loss: 0.0610
iter [30/30], Loss: 0.0594
iter [30/30], Loss: 0.0618
i

In [10]:
# 释放gpu显存
torch.cuda.empty_cache()

![loss](img/loss.png)